In [2]:
import numpy as np
from numpy import genfromtxt

import csv
import os

import deepSI
from deepSI import System_data

import pysindy
from pysindy import SINDy

from torch import nn
import torch

import matplotlib.pyplot as plt

from scipy.io import loadmat

from deepSI.utils import simple_res_net, feed_forward_nn

from utils import SINDy_model

c:\Users\20173928\Miniconda3\envs\sindy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading data

In [5]:
# data dir and file
save_dir = r"C:\Users\20173928\OneDrive - TU Eindhoven\Documents\Master\thesis\mscth\data"
save_dir = os.path.join(save_dir,'SilverboxFiles')
out = loadmat(os.path.join(save_dir,'SNLS80mV.mat'))

# extract data
u, y = out['V1'][0], out['V2'][0]

# store in deepsi data object
train, test = System_data(u=u[40000:],y=y[40000:]), System_data(u=u[:40000],y=y[:40000])

# load model
# if it is custom model define f,e and h classes
SUBNET_model = deepSI.load_system(r'C:\Users\20173928\OneDrive - TU Eindhoven\Documents\Master\thesis\mscth\models\silverbox_v1')

DeepSi Notes

In [6]:
# apply experiment and save state
# note output is denormalized by model object state is not
test_sim = SUBNET_model.apply_experiment(test, save_state=True)

# get output and states
y_out = test_sim.y
x_out = test_sim.x

# get mean and std used to normalize
y_mu  = SUBNET_model.norm.y0
y_std = SUBNET_model.norm.ystd

# denormalize
x_out = x_out*y_std + y_mu

SINDy Notes

In [7]:
# custom class with basic SINDy implementation
fs = 610.35
dt = 1/fs

model = SINDy_model(
  x_data=train.y,
  u_data=train.u,
  x_test=test.y, # if not specified will use data_split to create test set
  u_test=test.u,
  data_split=None, # percentage of x_data/u_data that will be used for testing
  discrete_time=True,
  t_default=dt
)

model.fit_s()
predicted = model.predict_s()
simulated = model.simulate_s()